In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=7be6ab84cb12d8febff7e25b9142602b3654a678af0bfa4f53997afe9f11f309
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
import pyspark.sql.functions as F

In [5]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)

df = spark.read.csv(SparkFiles.get('home_sales_revised.csv'), sep = ',', header = True, inferSchema = True)
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [6]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')


In [12]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
spark.sql('''
SELECT date_built, ROUND(AVG(price), 2) AS avg_cost
FROM home_sales
WHERE bedrooms = 4
GROUP BY date_built
SORT BY date_built
''').show()


+----------+---------+
|date_built| avg_cost|
+----------+---------+
|      2010|296800.75|
|      2011| 302141.9|
|      2012|298233.42|
|      2013|299999.39|
|      2014|299073.89|
|      2015|307908.86|
|      2016|296050.24|
|      2017|296576.69|
+----------+---------+



In [13]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql('''
SELECT date_built, ROUND(AVG(price), 2) AS avg_cost
FROM home_sales
WHERE bedrooms = 3 AND
  bathrooms = 3
GROUP BY date_built
SORT BY date_built
''').show()


+----------+---------+
|date_built| avg_cost|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [14]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
spark.sql('''
SELECT date_built, ROUND(AVG(price), 2) AS avg_cost
FROM home_sales
WHERE bedrooms = 3 AND
  bathrooms = 3 AND
  floors = 2 AND
  sqft_living >= 2000
GROUP BY date_built
SORT BY date_built
''').show()


+----------+---------+
|date_built| avg_cost|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [24]:
# 6. What is the "view" rating for homes costing more than or equal to $350,000? Determine the run time for this query, and round off your answer to two decimal places.

start_time = time.time()

spark.sql('''
SELECT ROUND(AVG(view), 2) AS avg_view
FROM home_sales
WHERE price >= 350000
''').show()

print("--- %s seconds ---" % (time.time() - start_time))

+--------+
|avg_view|
+--------+
|   32.26|
+--------+

--- 0.3933265209197998 seconds ---


In [25]:
# 7. Cache the the temporary table home_sales.
spark.sql('cache table home_sales')

DataFrame[]

In [26]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [28]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql('''
SELECT ROUND(AVG(view), 2) AS avg_view
FROM home_sales
WHERE price >= 350000
''').show()

print("--- %s seconds ---" % (time.time() - start_time))


+--------+
|avg_view|
+--------+
|   32.26|
+--------+

--- 0.31626391410827637 seconds ---


In [29]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy('date_built').mode('overwrite').parquet('df_partitioned')

In [30]:
# 11. Read the parquet formatted data.
df_partitioned = spark.read.parquet('df_partitioned')

In [31]:
# 12. Create a temporary table for the parquet data.
df_partitioned.createOrReplaceTempView('partitioned_home_sales')

In [36]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql('''
SELECT ROUND(AVG(view), 2) AS avg_view
FROM partitioned_home_sales
WHERE price >= 350000
''').show()

print("--- %s seconds ---" % (time.time() - start_time))

+--------+
|avg_view|
+--------+
|   32.26|
+--------+

--- 0.7232093811035156 seconds ---


In [37]:
# 14. Uncache the home_sales temporary table.
spark.sql('uncache table home_sales')

DataFrame[]

In [38]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')


False